# Otras arquitecturas "avanzadas"

Hasta ahora basicamente hemos visto problemas de etiquetar documentos como un tipo de clase bastante definido, y normalmente como "documentos standalone", es decir, que en nuestras arquitecturas, *x* era documento solo, y la clase solía ser, me gusta/no me gusta, política/deporte/música, en fin, etiquetas en general.

En sequence to sequence, vimos arquitecturas que nos permetían generar un output, basándonos en dos inputs, un primer input que condicionaba la salida (encoder), y otro input que era el que queríamos encontrarnos en el output (decoder).

Imaginaros que ahora queremos tener arquitecturas que nos dado pares de frases nos dicen si estas frases más o menos significan lo mismo. O en el caso del primer uso de la arquitectura que implementaremos, si dada una premisa, la hypothesis es correcta. Es decir inferir si dado x, se da y.

A esto se le llama Natural Language Infernece.

<div align="center">
    ![](http://wx3.sinaimg.cn/large/006Fmjmcly1fhiby4jojuj31dk0r6qkz.jpg =600x)
</div>



## Natural Language Inference

[SNLI](https://nlp.stanford.edu/projects/snli/) es una página de stanford que se dedica a trackear el progreso hecho en este ámbito.

La arquitectura, empieza más o menos igual que el sequence to sequence, pero en lugar de encoder-decoder, usaremos 2 encoders (LSTMs) y luego lo moveremos todo para arriba, donde clasificaremos las siguientes 3 clases: 

* Entailment
* Neutral
* Contradiction

<div align="center">
    ![](https://ai2-s2-public.s3.amazonaws.com/figures/2017-08-08/29f34ca5778f742b2cee0dfc70819eca11759208/3-Figure1-1.png =400x)
</div>

Ahora que se esta empezando a luchar contra las fake-news con models de Machine Learning y Deep Learning, esta arquitectura tiene muchísimo sentido, pues por ejemplo podríamos pasar una notícia, y un seguido de "facts" y ver si la relación es de entailment o contradiction, y predecir si es fake-news o no.



## Imports

In [12]:
import numpy as np
from random import uniform, shuffle
from random import sample

## Data Generation

In [14]:
equals = []
for _ in range(1000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 1
    equals.append((x0, x1, np.array([1, 0, 0])))

for _ in range(1000):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 0
    equals.append((x0, x1, np.array([1, 0, 0])))
            

In [15]:
meh = []
for _ in range(2000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.50:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.50:
            x1[ix] = 1
    meh.append((x0, x1, np.array([0, 1, 0])))


In [16]:
different = []
for _ in range(1000):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.80:
            x1[ix] = 1
    different.append((x0, x1,  np.array([0, 0, 1])))

for _ in range(1000):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.8:
            x1[ix] = 0
    different.append((x0, x1, np.array([0, 0, 1])))

## Model Definition

<div align="center">
    ![](https://i.imgur.com/g8jzlfx.png =450x)
</div>


In [17]:
from keras.layers import Dense, CuDNNLSTM, Dropout, Embedding, Input, Concatenate, Average, Multiply, Add, LSTM
from keras.models import Model
import keras.backend as K

In [18]:
p = Input(shape=(100, ))
h = Input(shape=(100, ))

emb_layer = Embedding(input_dim=2, output_dim=100, input_length=100)
emb_p = emb_layer(p)#(p)
emb_h = emb_layer(h)#(h)

plstm = CuDNNLSTM(100)(emb_p)
hlstm = CuDNNLSTM(100)(emb_h)

# Multiply()([plstm,hlstm]), Add()([plstm,hlstm]), Average()([plstm, hlstm])

ph = Concatenate()([plstm, hlstm, Multiply()([plstm,hlstm]), Add()([plstm,hlstm]), Average()([plstm, hlstm])])

d_1 = Dense(100, activation='relu')(ph)
d_2 = Dense(100, activation='relu')(d_1)
out = Dense(3, activation='softmax')(d_2)

model = Model(inputs=[p, h], outputs=out)

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 100)     200         input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)        (None, 100)          80800       embedding_2[0][0]                
__________

## Dataset Preparation

In [20]:
dataset = equals+meh+different
shuffle(dataset)

In [21]:
for s in sample(dataset, 3):
    x, xx, y = s
    print(np.sum(x), np.sum(xx), y)

46.0 61.0 [0 1 0]
29.0 28.0 [1 0 0]
56.0 45.0 [0 1 0]


In [22]:
x0, x1, y = zip(*dataset)

x0 = np.vstack(x0).reshape(len(x0), -1)
x1 = np.vstack(x1).reshape(len(x0), -1)
y = np.vstack(y)

x0.shape, x1.shape, y.shape

((6000, 100), (6000, 100), (6000, 3))

## Fit model

In [23]:
model.fit(x=[x0, x1], y=y, epochs=20)

Epoch 1/20
6000/6000 [==============================] - 12s 2ms/step - loss: 0.2720 - acc: 0.9028
Epoch 2/20
6000/6000 [==============================] - 9s 2ms/step - loss: 0.1050 - acc: 0.9707
Epoch 3/20
6000/6000 [==============================] - 9s 1ms/step - loss: 0.0890 - acc: 0.9798
Epoch 4/20
5536/6000 [==========================>...] - ETA: 0s - loss: 0.0606 - acc: 0.9848

6000/6000 [==============================] - 9s 1ms/step - loss: 0.0597 - acc: 0.9847
Epoch 5/20
6000/6000 [==============================] - 9s 1ms/step - loss: 0.0458 - acc: 0.9885
Epoch 6/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0437 - acc: 0.9898
Epoch 7/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0352 - acc: 0.9918
Epoch 8/20
1952/6000 [========>.....................] - ETA: 5s - loss: 0.0871 - acc: 0.9872

6000/6000 [==============================] - 8s 1ms/step - loss: 0.0454 - acc: 0.9920
Epoch 9/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0353 - acc: 0.9922
Epoch 10/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0247 - acc: 0.9943
Epoch 11/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0511 - acc: 0.9912
Epoch 12/20
1184/6000 [====>.........................] - ETA: 6s - loss: 8.0527e-04 - acc: 1.0000

6000/6000 [==============================] - 8s 1ms/step - loss: 0.0436 - acc: 0.9918
Epoch 13/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0160 - acc: 0.9962
Epoch 14/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0151 - acc: 0.9970
Epoch 15/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0162 - acc: 0.9957
Epoch 16/20
1088/6000 [====>.........................] - ETA: 6s - loss: 0.0315 - acc: 0.9954

6000/6000 [==============================] - 8s 1ms/step - loss: 0.0168 - acc: 0.9963
Epoch 17/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0144 - acc: 0.9963
Epoch 18/20
6000/6000 [==============================] - 8s 1ms/step - loss: 0.0187 - acc: 0.9958
Epoch 19/20
6000/6000 [==============================] - 9s 1ms/step - loss: 0.0260 - acc: 0.9948
Epoch 20/20
1056/6000 [====>.........................] - ETA: 7s - loss: 1.7995e-04 - acc: 1.0000

6000/6000 [==============================] - 8s 1ms/step - loss: 0.0075 - acc: 0.9982


In [24]:
eq = sample(equals, k=1)

In [25]:
for eq in sample(dataset, k=3):
    print(np.sum(eq[0].reshape(1,-1)), np.sum(eq[1].reshape(1,-1)))
    print(model.predict(x=[eq[0].reshape(1,-1), eq[1].reshape(1,-1)]).round())

75.0 79.0
[[1. 0. 0.]]
74.0 23.0
[[0. 0. 1.]]
66.0 82.0
[[1. 0. 0.]]


## Evaluate Model

In [26]:
test_set = []
for _ in range(100):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 1
    test_set.append((x0, x1, np.array([1, 0, 0])))

for _ in range(100):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.25:
            x1[ix] = 0
    test_set.append((x0, x1, np.array([1, 0, 0])))

for _ in range(200):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.50:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.50:
            x1[ix] = 1
    test_set.append((x0, x1, np.array([0, 1, 0])))
    
for _ in range(100):
    x0 = np.zeros(100)
    x1 = np.zeros(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 1
        if uniform(0.0, 1.0)>0.80:
            x1[ix] = 1
    test_set.append((x0, x1,  np.array([0, 0, 1])))

for _ in range(100):
    x0 = np.ones(100)
    x1 = np.ones(100)
    for ix in range(100):
        if uniform(0.0, 1.0)>0.25:
            x0[ix] = 0
        if uniform(0.0, 1.0)>0.8:
            x1[ix] = 0
    test_set.append((x0, x1, np.array([0, 0, 1])))
    

shuffle(test_set)
    
xt0, xt1, yt = zip(*test_set)

xt0 = np.vstack(xt0).reshape(len(xt0), -1)
xt1 = np.vstack(xt1).reshape(len(xt0), -1)
yt = np.vstack(yt)

xt0.shape, xt1.shape, yt.shape

((600, 100), (600, 100), (600, 3))

In [27]:
model.evaluate(x=[xt0, xt1], y=yt)

600/600 [==============================] - 0s 754us/step


[0.08574207445937645, 0.9883333333333333]